In [57]:
# Import libraries
import numpy
from matplotlib import pyplot
from pandas import read_table
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kylek\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [59]:
# Load dataset
filepath = 'Data\SMSSpamCollection.txt'
data = read_csv(filepath, sep='[\t]', names=['class', 'text'], engine='python')

In [60]:
data.shape

(5574, 2)

In [61]:
data.head(20)

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [29]:
data.tail(20)

,class,text
5554,ham,Have a safe trip to Nigeria. Wish you happines...
5555,ham,Hahaha..use your brain dear
5556,ham,Well keep in mind I've only got enough gas for...
5557,ham,Yeh. Indians was nice. Tho it did kane me off ...
5558,ham,Yes i have. So that's why u texted. Pshew...mi...
5559,ham,No. I meant the calculation is the same. That ...
5560,ham,"Sorry, I'll call later"
5561,ham,if you aren't here in the next &lt;#&gt; hou...
5562,ham,Anything lor. Juz both of us lor.
5563,ham,Get me out of this dump heap. My mom decided t...


In [30]:
print(data.groupby('class').size())

class
ham     4827
spam     747
dtype: int64


In [31]:
data.isnull().sum()

class    0
text     0
dtype: int64

In [46]:
# Get counts of unique characters:
char_counts = {}
for message in data['text']:
        for char in message:
            char_counts[char] = 1 if char not in char_counts.keys() else char_counts[char] + 1
                
print("Number of unique characters: %s" % len(char_counts.values()))
print("\nCounts of unique characters:")
print(sorted(char_counts.items(), key = lambda kv:kv[1]))

Number of unique characters: 116

Counts of unique characters:
[('ú', 1), ('è', 1), ('^', 1), ('»', 1), ('—', 1), ('É', 1), ('ì', 1), ('鈥', 1), ('┾', 1), ('〨', 1), ('“', 2), ('\x91', 2), ('\x93', 2), ('¡', 2), ('~', 3), ('’', 3), ('\x96', 3), ('é', 3), ('\\', 4), ('\x94', 4), ('[', 5), (']', 5), ('<', 6), ('–', 9), ('%', 10), ('_', 15), ('…', 16), ('$', 23), ('>', 31), ('Z', 33), ('|', 36), ('‘', 37), ('@', 38), ('\x92', 39), ('=', 47), ('Ü', 53), ('Q', 73), ('(', 119), ('ü', 120), ('+', 137), ('q', 163), ('#', 287), ('J', 308), ('*', 311), ('X', 319), ('£', 329), ('"', 347), ('V', 367), ('K', 404), ('/', 419), ('z', 472), (')', 499), ('-', 585), ('j', 640), ('F', 651), ('9', 665), (':', 745), (';', 768), ('6', 806), ('3', 852), ('7', 858), ('G', 880), ('B', 894), ('&', 922), ('P', 987), ('4', 1057), ('Y', 1064), ('5', 1073), ('L', 1125), ('U', 1138), ('R', 1154), ('D', 1157), ('M', 1229), ('x', 1298), ('8', 1315), ('W', 1374), ('!', 1397), ('C', 1405), ('H', 1549), ('?', 1550), ('N', 

In [63]:
# Remove stop words and stem:
stopwords_list = stopwords.words('english')
ps = PorterStemmer()
cleaned_messages = []
for i in range(0, len(data['text'])):
    message = data['text'][i].lower().split()
    message = [ps.stem(word) for word in message if not word in stopwords_list]
    message = ' '.join(message)
    cleaned_messages.append(message)

In [64]:
print(cleaned_messages)

['go jurong point, crazy.. avail bugi n great world la e buffet... cine got amor wat...', 'ok lar... joke wif u oni...', "free entri 2 wkli comp win fa cup final tkt 21st may 2005. text fa 87121 receiv entri question(std txt rate)t&c' appli 08452810075over18'", 'u dun say earli hor... u c alreadi say...', 'nah think goe usf, live around though', "freemsg hey darl 3 week' word back! i'd like fun still? tb ok! xxx std chg send, £1.50 rcv", 'even brother like speak me. treat like aid patent.', "per request 'mell mell (oru minnaminungint nurungu vettam)' set callertun callers. press *9 copi friend callertun", 'winner!! valu network custom select receivea £900 prize reward! claim call 09061701461. claim code kl341. valid 12 hour only.', 'mobil 11 month more? u r entitl updat latest colour mobil camera free! call mobil updat co free 08002986030', "i'm gonna home soon want talk stuff anymor tonight, k? i'v cri enough today.", 'six chanc win cash! 100 20,000 pound txt> csh11 send 87575. cost 1